In [2]:
# Importing re package for using regular expressions 
import re
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os.path

# Download the csv file from Mitre. Future enhancement could be to compare checksum to determine whether to download new version.
wgetcvedownload = 'https://cve.mitre.org/data/downloads/allitems.csv'

if os.path.isfile('/home/ec2-user/SageMaker/notebooks/tools/allitems.csv'):
    print("File already exists. No need to download.")
else:
    print('Need to download')
    ! wget {wgetcvedownload}
    ! mv 'allitems.csv' tools/

File already exists. No need to download.


In [3]:
dfCVE = pd.read_csv('tools/allitems.csv', sep=',', skiprows=[0,1,3,4,5,6,7,8,9], header=0, encoding = "ISO-8859-1")

# Function to clean the names - there is probably a better way of doing all of this.
def normalize_columns(Phase): 
    Phase = str(Phase)
    if re.search('(\()(\d\d\d\d\d\d\d\d)', Phase): 
        pos = re.search('(\()(\d\d\d\d\d\d\d\d)', Phase)
        pos = pos.group(2)
        date_str = pos
        format_str = '%Y%m%d'
        pos = datetime.datetime.strptime(date_str, format_str)      
        return pos
    else: 
        # if cleanup needed return the same name 
        pos = "19000101"
        date_str = pos
        format_str = '%Y%m%d'
        pos = datetime.datetime.strptime(date_str, format_str) 
        return 'null'

# Add a normalized CVE date column
dfCVE['CVEDate'] = dfCVE['Phase'].apply(normalize_columns)
# Drop the null date values
dfCVE.drop(dfCVE[dfCVE.CVEDate == 'null'].index, inplace=True)
dfCVE

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Name,Status,Description,References,Phase,Votes,Comments,CVEDate
0,CVE-1999-0001,Candidate,ip_input.c in BSD-derived TCP/IP implementatio...,BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 - ...,Modified (20051217),"MODIFY(1) Frech | NOOP(2) Northcutt, W...",Christey> A Bugtraq posting indicates that the...,2005-12-17 00:00:00
3,CVE-1999-0004,Candidate,"MIME buffer overflow in email clients, e.g. So...",CERT:CA-98.10.mime_buffer_overflows | MS:M...,Modified (19990621),"ACCEPT(8) Baker, Cole, Collins, Dik, Landfi...","Frech> Extremely minor, but I believe e-mail i...",1999-06-21 00:00:00
14,CVE-1999-0015,Candidate,Teardrop IP denial of service.,CERT:CA-97.28.Teardrop_Land | OVAL:oval:or...,Modified (20090302),ACCEPT(1) Wall | MODIFY(1) Frech | ...,Frech> XF: teardrop-mod | Christey> Not su...,2009-03-02 00:00:00
19,CVE-1999-0020,Candidate,** REJECT ** DO NOT USE THIS CANDIDATE NUMBER...,NaN,Modified (20050204),"MODIFY(1) Frech | NOOP(4) Levy, Northc...",Frech> XF:lpr-bo | Christey> DUPE CVE-1999...,2005-02-04 00:00:00
29,CVE-1999-0030,Candidate,root privileges via buffer overflow in xlock c...,AUSCERT:AA-97.24.IRIX.xlock.buffer.overflow.vu...,Proposed (19990623),"ACCEPT(3) Levy, Ozancin, Prosser | NOO...",Frech> XF:xlock-bo (also add) | As per xlo...,1999-06-23 00:00:00
32,CVE-1999-0033,Candidate,Command execution in Sun systems via buffer ov...,CERT:CA-97.18.at | SUN:00160 | XF:sun-...,Modified (20040811),"ACCEPT(8) Baker, Cole, Collins, Dik, Hill, ...",Frech> This vulnerability also manifests itsel...,2004-08-11 00:00:00
60,CVE-1999-0061,Candidate,"File creation and deletion, and remote executi...",NAI:NAI-20 | XF:bsd-lpd,Proposed (19990630),"ACCEPT(3) Frech, Hill, Northcutt | REC...",Christey> This should be split into three sepa...,1999-06-30 00:00:00
75,CVE-1999-0076,Candidate,Buffer overflow in wu-ftp from PASV command ca...,XF:ftp-args,Modified (19990925),"ACCEPT(3) Baker, Frech, Ozancin | NOOP...",Balinsky> Don't know what this is. Is this th...,1999-09-25 00:00:00
77,CVE-1999-0078,Candidate,pcnfsd (aka rpc.pcnfsd) allows local users to ...,CERT:CA-96.08.pcnfsd | XF:rpc-pcnfsd,Modified (19990621),"ACCEPT(5) Collins, Frech, Landfield, Northc...","Christey> This candidate should be SPLIT, sinc...",1999-06-21 00:00:00
85,CVE-1999-0086,Candidate,AIX routed allows remote users to modify sensi...,ERS:ERS-SVA-E01-1998:001.1 | XF:ibm-routed,Interim (19990630),"ACCEPT(2) Northcutt, Shostack | MODIFY...",Frech> Reference: XF:ibm-routed | Prosser>...,1999-06-30 00:00:00


In [4]:
# How many per year
CVEYear = dfCVE['CVEDate'].groupby([dfCVE.CVEDate.dt.year]).agg('count').to_frame('counts')

# Convert results to dataframe
dfCVEYear = pd.DataFrame(CVEYear)
dfCVEYear = dfCVEYear.reset_index()
dfCVEYear.columns = ['Year', 'Count'] # change column names
dfCVEYear

,Year,Count
0,1999,249
1,2000,351
2,2001,713
3,2002,881
4,2003,1053
5,2004,1234
6,2005,7274
7,2006,7142
8,2007,7731
9,2008,6026


In [5]:
# Which months are busiest for CVE - Could indicate more patching
CVEMonth = dfCVE['CVEDate'].groupby([dfCVE.CVEDate.dt.month]).agg('count').to_frame('counts')
# Convert results to dataframe
dfCVEMonth = pd.DataFrame(CVEMonth)
dfCVEMonth = dfCVEMonth.reset_index()
dfCVEMonth.columns = ['Month', 'Count'] # change column names
dfCVEMonth

,Month,Count
0,1,20184
1,2,12733
2,3,14568
3,4,12006
4,5,11882
5,6,13866
6,7,11862
7,8,13621
8,9,12698
9,10,12492


In [6]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

# Confirm that Bokeh is working
output_notebook()
dfCVEYearMod = dfCVEYear.iloc[1:]
p = figure(plot_width=400, plot_height=400)
p.circle(dfCVEYearMod['Year'],dfCVEYearMod['Count'], size=10, line_color="navy", fill_color="red", fill_alpha=0.5)

show(p) # show the results

Loading BokehJS ...